In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
# 1. Đọc dữ liệu từ file CSV
df = pd.read_csv("D:\\Python\\Project2_phishing\\Dataset.csv")

In [3]:
# Lấy danh sách các feature và nhãn
features = df.columns[1:].tolist()
X = df[features].values
y = df["Type"].values

In [4]:
# 2. Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [5]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
                
    def is_leaf_node(self):
        return self.value is not None

In [ ]:
class DecisionTreeGini:
    def __init__(self, min_samples_split=4, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        #kiểm tra điều kiện dừng
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        #chọn đặc trưng tốt nhất
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        #tạo cây con
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gini = float('inf')
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gini = self._gini_impurity(y, X_column, thr)
                if gini < best_gini:
                    best_gini = gini
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    #tính total gini-impurity
    def _gini_impurity(self, y, X_column, threshold):
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 1

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        gini_left = self._gini(y[left_idxs])
        gini_right = self._gini(y[right_idxs])
        return (n_l/n) * gini_left + (n_r/n) * gini_right

    #chia dữ liệu thành 2 nhánh
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten() #chuyển về mảng 1 chiều
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    #tính gini-impurity cho 1 node lá
    def _gini(self, y):
        hist = np.bincount(y) #Đếm số lượng mẫu mỗi class
        probs = hist / len(y)
        return 1 - np.sum(probs ** 2)

    #trả về nhãn phổ biến nhất
    def _most_common_label(self, y):
        counter = Counter(y) #đếm số lần xuất hiện của mỗi giá trị trong mảng y
        return counter.most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [15]:
# 3. Huấn luyện mô hình cây quyết định sử dụng Gini Impurity
dt_model = DecisionTreeGini(max_depth=4)
dt_model.fit(X_train, y_train)

In [16]:
# 4. Dự đoán trên tập kiểm tra
y_preds = dt_model.predict(X_valid)

In [17]:
# 11. Dự đoán và hiển thị kết quả
print(pd.DataFrame({'y': y_valid, 'y_preds': y_preds}))

       y  y_preds
0      1        0
1      0        0
2      0        0
3      1        1
4      1        1
...   ..      ...
49585  0        0
49586  1        1
49587  0        0
49588  0        1
49589  0        0

[49590 rows x 2 columns]


In [18]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_valid, y_preds))


Confusion Matrix:
[[22585  2998]
 [ 6934 17073]]


In [19]:
print("\nClassification Report:")
print(classification_report(y_valid, y_preds))


Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.88      0.82     25583
           1       0.85      0.71      0.77     24007

    accuracy                           0.80     49590
   macro avg       0.81      0.80      0.80     49590
weighted avg       0.81      0.80      0.80     49590

